# Assignment 4 - Predictive Policing
## Author - Salinee Kingbaisomboon
### UW NetID: 1950831

## Instructions
1. Read data.
2. Apply three techniques for filter selection: Filter methods, Wrapper methods, Embedded methods.
3. Describe your findings.

In [100]:
# Load necessary libraries
import pandas as pd
import numpy as np

from sklearn.metrics import mutual_info_score
from sklearn.feature_selection import RFE # Recursive Feature Elimination (for backward model selection)
from sklearn import linear_model # LASSO
from sklearn.linear_model import LinearRegression
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings("ignore") # To suppress warning

%matplotlib inline

pd.options.display.max_rows = None

# Declare Functions used in this assignment

In [38]:
# Function to replace missing numeric values
def replace_missing_value(x, col):
    # We determine the locations of the question marks
    QuestionMark = x.loc[:, col].astype(str) == "?"
    # If there are question marks
    if sum(QuestionMark) > 0:
        # Convert the current column to numeric data including nans
        x.loc[:, col] = x.apply(lambda y: pd.to_numeric(y, errors='coerce'), axis=0)
        # Get the Nan array from the current column
        HasNan = np.isnan(x.loc[:, col]) 
        # Calculate the Median for current column without Nan
        Median = np.nanmedian(x.loc[:, col])
        # Replace the missing value with Median
        x.loc[HasNan, col] = Median
    return x.loc[:, col]

# Read and perform data cleaning

In [28]:
# Load data
filename = 'http://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.data'
df = pd.read_csv(filename, header=None)
# Headers from https://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.names
df.columns = ['state','county','community','communityname','fold','population','householdsize','racepctblack','racePctWhite','racePctAsian',
              'racePctHisp','agePct12t21','agePct12t29','agePct16t24','agePct65up','numbUrban','pctUrban','medIncome','pctWWage',
              'pctWFarmSelf','pctWInvInc','pctWSocSec','pctWPubAsst','pctWRetire','medFamInc','perCapInc','whitePerCap','blackPerCap',
              'indianPerCap','AsianPerCap','OtherPerCap','HispPerCap','NumUnderPov','PctPopUnderPov','PctLess9thGrade','PctNotHSGrad',
              'PctBSorMore','PctUnemployed','PctEmploy','PctEmplManu','PctEmplProfServ','PctOccupManu','PctOccupMgmtProf','MalePctDivorce',
              'MalePctNevMarr','FemalePctDiv','TotalPctDiv','PersPerFam','PctFam2Par','PctKids2Par','PctYoungKids2Par','PctTeen2Par','PctWorkMomYoungKids',
              'PctWorkMom','NumIlleg','PctIlleg','NumImmig','PctImmigRecent','PctImmigRec5','PctImmigRec8','PctImmigRec10','PctRecentImmig',
              'PctRecImmig5','PctRecImmig8','PctRecImmig10','PctSpeakEnglOnly','PctNotSpeakEnglWell','PctLargHouseFam','PctLargHouseOccup',
              'PersPerOccupHous','PersPerOwnOccHous','PersPerRentOccHous','PctPersOwnOccup','PctPersDenseHous','PctHousLess3BR','MedNumBR',
              'HousVacant','PctHousOccup','PctHousOwnOcc','PctVacantBoarded','PctVacMore6Mos','MedYrHousBuilt','PctHousNoPhone','PctWOFullPlumb',
              'OwnOccLowQuart','OwnOccMedVal','OwnOccHiQuart','RentLowQ','RentMedian','RentHighQ','MedRent','MedRentPctHousInc','MedOwnCostPctInc',
              'MedOwnCostPctIncNoMtg','NumInShelters','NumStreet','PctForeignBorn','PctBornSameState','PctSameHouse85','PctSameCity85',
              'PctSameState85','LemasSwornFT','LemasSwFTPerPop','LemasSwFTFieldOps','LemasSwFTFieldPerPop','LemasTotalReq','LemasTotReqPerPop',
              'PolicReqPerOffic','PolicPerPop','RacialMatchCommPol','PctPolicWhite','PctPolicBlack','PctPolicHisp','PctPolicAsian','PctPolicMinor',
              'OfficAssgnDrugUnits','NumKindsDrugsSeiz','PolicAveOTWorked','LandArea','PopDens','PctUsePubTrans','PolicCars','PolicOperBudg',
              'LemasPctPolicOnPatr','LemasGangUnitDeploy','LemasPctOfficDrugUn','PolicBudgPerPop','ViolentCrimesPerPop'
             ]

Based on https://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.names, under the **Attribute Information** section.

<font color=blue>**ViolentCrimesPerPop:**</font> total number of violent crimes per 100K popuation (numeric - decimal) GOAL attribute (to be predicted)

**Note:** Based on https://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.names, All numeric data was normalized into the decimal range 0.00-1.00 using an Unsupervised, equal-interval binning method. Therefore, we don't need to perform the **Normalization** with this data set.

# Drop the unused columns

- <font color=red>**state:**</font> US state (by number) - not counted as predictive above, but if considered, should be consided nominal (nominal)
- <font color=red>**county:**</font> numeric code for county - not predictive, and many missing values (numeric)
- <font color=red>**community:**</font> numeric code for community - not predictive and many missing values (numeric)
- <font color=red>**communityname:**</font> community name - not predictive - for information only (string)
- <font color=red>**fold:**</font> fold number for non-random 10 fold cross validation, potentially useful for debugging, paired tests - not predictive (numeric)

In [29]:
df.drop(['state'],axis=1, inplace=True)
df.drop(['county'],axis=1, inplace=True)
df.drop(['community'],axis=1, inplace=True)
df.drop(['communityname'],axis=1, inplace=True)
df.drop(['fold'],axis=1, inplace=True)

# Replace missing values with Median on each columns (except ViolentCrimesPerPop - Goal attribute)
After data's exploration, we found that there are multiple of columns have missinge values. Based on https://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.names, LEMAS survey was of the police departments with at least 100 officers, plus a random sample of smaller departments. Many communities are missing 
LEMAS data.

The remedy for I use is to **replace missing value with Median** for each columns without NaN.

In [41]:
# Columns which have nan value such as ?
for col in ['OtherPerCap','LemasSwornFT','LemasSwFTPerPop','LemasSwFTFieldOps','LemasSwFTFieldPerPop','LemasTotalReq',
            'LemasTotReqPerPop','PolicReqPerOffic','PolicPerPop','RacialMatchCommPol','PctPolicWhite','PctPolicBlack',
            'PctPolicHisp','PctPolicAsian','PctPolicMinor','OfficAssgnDrugUnits','NumKindsDrugsSeiz','PolicAveOTWorked',
            'PolicCars','PolicOperBudg','LemasPctPolicOnPatr','LemasGangUnitDeploy','PolicBudgPerPop']:
    # Replace missing values with Median for current column without Nan
    df.loc[:, col] = replace_missing_value(df, col)

In [43]:
# View first five rows of the data frame
df.head()

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,pctWFarmSelf,pctWInvInc,pctWSocSec,pctWPubAsst,pctWRetire,medFamInc,perCapInc,whitePerCap,blackPerCap,indianPerCap,AsianPerCap,OtherPerCap,HispPerCap,NumUnderPov,PctPopUnderPov,PctLess9thGrade,PctNotHSGrad,PctBSorMore,PctUnemployed,PctEmploy,PctEmplManu,PctEmplProfServ,PctOccupManu,PctOccupMgmtProf,MalePctDivorce,MalePctNevMarr,FemalePctDiv,TotalPctDiv,PersPerFam,PctFam2Par,PctKids2Par,PctYoungKids2Par,PctTeen2Par,PctWorkMomYoungKids,PctWorkMom,NumIlleg,PctIlleg,NumImmig,PctImmigRecent,PctImmigRec5,PctImmigRec8,PctImmigRec10,PctRecentImmig,PctRecImmig5,PctRecImmig8,PctRecImmig10,PctSpeakEnglOnly,PctNotSpeakEnglWell,PctLargHouseFam,PctLargHouseOccup,PersPerOccupHous,PersPerOwnOccHous,PersPerRentOccHous,PctPersOwnOccup,PctPersDenseHous,PctHousLess3BR,MedNumBR,HousVacant,PctHousOccup,PctHousOwnOcc,PctVacantBoarded,PctVacMore6Mos,MedYrHousBuilt,PctHousNoPhone,PctWOFullPlumb,OwnOccLowQuart,OwnOccMedVal,OwnOccHiQuart,RentLowQ,RentMedian,RentHighQ,MedRent,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LemasSwornFT,LemasSwFTPerPop,LemasSwFTFieldOps,LemasSwFTFieldPerPop,LemasTotalReq,LemasTotReqPerPop,PolicReqPerOffic,PolicPerPop,RacialMatchCommPol,PctPolicWhite,PctPolicBlack,PctPolicHisp,PctPolicAsian,PctPolicMinor,OfficAssgnDrugUnits,NumKindsDrugsSeiz,PolicAveOTWorked,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,0.20,1.0,0.37,0.72,0.34,0.60,0.29,0.15,0.43,0.39,0.40,0.39,0.32,0.27,0.27,0.36,0.41,0.08,0.19,0.10,0.18,0.48,0.27,0.68,0.23,0.41,0.25,0.52,0.68,0.40,0.75,0.75,0.35,0.55,0.59,0.61,0.56,0.74,0.76,0.04,0.14,0.03,0.24,0.27,0.37,0.39,0.07,0.07,0.08,0.08,0.89,0.06,0.14,0.13,0.33,0.39,0.28,0.55,0.09,0.51,0.5,0.21,0.71,0.52,0.05,0.26,0.65,0.14,0.06,0.22,0.19,0.18,0.36,0.35,0.38,0.34,0.38,0.46,0.25,0.04,0.0,0.12,0.42,0.50,0.51,0.64,0.03,0.13,0.96,0.17,0.06,0.18,0.44,0.13,0.94,0.93,0.03,0.07,0.1,0.07,0.02,0.57,0.29,0.12,0.26,0.20,0.06,0.04,0.90,0.5,0.32,0.14,0.20
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,0.02,1.0,0.31,0.72,0.11,0.45,0.25,0.29,0.39,0.29,0.37,0.38,0.33,0.16,0.30,0.22,0.35,0.01,0.24,0.14,0.24,0.30,0.27,0.73,0.57,0.15,0.42,0.36,1.00,0.63,0.91,1.00,0.29,0.43,0.47,0.60,0.39,0.46,0.53,0.00,0.24,0.01,0.52,0.62,0.64,0.63,0.25,0.27,0.25,0.23,0.84,0.10,0.16,0.10,0.17,0.29,0.17,0.26,0.20,0.82,0.0,0.02,0.79,0.24,0.02,0.25,0.65,0.16,0.00,0.21,0.20,0.21,0.42,0.38,0.40,0.37,0.29,0.32,0.18,0.00,0.0,0.21,0.50,0.34,0.60,0.52,0.02,0.18,0.97,0.21,0.04,0.17,0.29,0.18,0.74,0.78,0.12,0.06,0.0,0.20,0.04,0.57,0.26,0.02,0.12,0.45,0.08,0.03,0.75,0.5,0.00,0.15,0.67
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,0.00,0.0,0.30,0.58,0.19,0.39,0.38,0.40,0.84,0.28,0.27,0.29,0.27,0.07,0.29,0.28,0.39,0.01,0.27,0.27,0.43,0.19,0.36,0.58,0.32,0.29,0.49,0.32,0.63,0.41,0.71,0.70,0.45,0.42,0.44,0.43,0.43,0.71,0.67,0.01,0.46,0.00,0.07,0.06,0.15,0.19,0.02,0.02,0.04,0.05,0.88,0.04,0.20,0.20,0.46,0.52,0.43,0.42,0.15,0.51,0.5,0.01,0.86,0.41,0.29,0.30,0.52,0.47,0.45,0.18,0.17,0.16,0.27,0.29,0.27,0.31,0.48,0.39,0.28,0.00,0.0,0.14,0.49,0.54,0.67,0.56,0.02,0.18,0.97,0.21,0.04,0.17,0.29,0.18,0.74,0.78,0.12,0.06,0.0,0.20,0.04,0.57,0.26,0.01,0.21,0.02,0.08,0.03,0.75,0.5,0.00,0.15,0.43
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,0.06,1.0,0.58,0.89,0.21,0.43,0.36,0.20,0.82,0.51,0.36,0.40,0.39,0.16,0.25,0.36,0.44,0.01,0.10,0.09,0.25,0.31,0.33,0.71,0.36,0.45,0.37,0.39,0.34,0.45,0.49,0.44,0.75,0.65,0.54,0.83,0.65,0.85,0.86,0.03,0.33,0.02,0.11,0.20,0.30,0.31,0.05,0.08,0.11,0.11,0.81,0.08,0.56,0.62,0.85,0.77,1.00,0.94,0.12,0.01,0.5,0.01,0.97,0.96,0.60,0.47,0.52,0.11,0.11,0.24,0.21,0.19,0.75,0.70,0.77,0.89,0.63,0.51,0.47,0.00,0.0,0.19

In [44]:
# Print DataFrame's size
print(df.shape)
# Print DataFrame's data types
# Note: we can see that all columns were numeric columns now (after did the missing value replacement)
print(df.dtypes)

(1994, 123)
population               float64
householdsize            float64
racepctblack             float64
racePctWhite             float64
racePctAsian             float64
racePctHisp              float64
agePct12t21              float64
agePct12t29              float64
agePct16t24              float64
agePct65up               float64
numbUrban                float64
pctUrban                 float64
medIncome                float64
pctWWage                 float64
pctWFarmSelf             float64
pctWInvInc               float64
pctWSocSec               float64
pctWPubAsst              float64
pctWRetire               float64
medFamInc                float64
perCapInc                float64
whitePerCap              float64
blackPerCap              float64
indianPerCap             float64
AsianPerCap              float64
OtherPerCap              float64
HispPerCap               float64
NumUnderPov              float64
PctPopUnderPov           float64
PctLess9thGrade          float6

In [46]:
# Described DataFrame
df.describe()

,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,agePct12t29,agePct16t24,agePct65up,numbUrban,pctUrban,medIncome,pctWWage,pctWFarmSelf,pctWInvInc,pctWSocSec,pctWPubAsst,pctWRetire,medFamInc,perCapInc,whitePerCap,blackPerCap,indianPerCap,AsianPerCap,OtherPerCap,HispPerCap,NumUnderPov,PctPopUnderPov,PctLess9thGrade,PctNotHSGrad,PctBSorMore,PctUnemployed,PctEmploy,PctEmplManu,PctEmplProfServ,PctOccupManu,PctOccupMgmtProf,MalePctDivorce,MalePctNevMarr,FemalePctDiv,TotalPctDiv,PersPerFam,PctFam2Par,PctKids2Par,PctYoungKids2Par,PctTeen2Par,PctWorkMomYoungKids,PctWorkMom,NumIlleg,PctIlleg,NumImmig,PctImmigRecent,PctImmigRec5,PctImmigRec8,PctImmigRec10,PctRecentImmig,PctRecImmig5,PctRecImmig8,PctRecImmig10,PctSpeakEnglOnly,PctNotSpeakEnglWell,PctLargHouseFam,PctLargHouseOccup,PersPerOccupHous,PersPerOwnOccHous,PersPerRentOccHous,PctPersOwnOccup,PctPersDenseHous,PctHousLess3BR,MedNumBR,HousVacant,PctHousOccup,PctHousOwnOcc,PctVacantBoarded,PctVacMore6Mos,MedYrHousBuilt,PctHousNoPhone,PctWOFullPlumb,OwnOccLowQuart,OwnOccMedVal,OwnOccHiQuart,RentLowQ,RentMedian,RentHighQ,MedRent,MedRentPctHousInc,MedOwnCostPctInc,MedOwnCostPctIncNoMtg,NumInShelters,NumStreet,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LemasSwornFT,LemasSwFTPerPop,LemasSwFTFieldOps,LemasSwFTFieldPerPop,LemasTotalReq,LemasTotReqPerPop,PolicReqPerOffic,PolicPerPop,RacialMatchCommPol,PctPolicWhite,PctPolicBlack,PctPolicHisp,PctPolicAsian,PctPolicMinor,OfficAssgnDrugUnits,NumKindsDrugsSeiz,PolicAveOTWorked,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
count,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000,1994.000000
mean,0.057593,0.463395,0.179629,0.753716,0.153681,0.144022,0.424218,0.493867,0.336264,0.423164,0.064072,0.696269,0.361123,0.558154,0.291570,0.495687,0.471133,0.317778,0.479248,0.375677,0.350251,0.368049,0.291098,0.203506,0.322357,0.284724,0.386279,0.055507,0.303024,0.315807,0.383330,0.361675,0.363531,0.501073,0.396384,0.440597,0.391224,0.441339,0.461244,0.434453,0.487568,0.494273,0.487748,0.610918,0.620657,0.664032,0.582884,0.501449,0.526690,0.036294,0.249995,0.030060,0.320211,0.360622,0.399077,0.427879,0.181364,0.182126,0.184774,0.182879,0.785903,0.150587,0.267608,0.251891,0.462101,0.494428,0.404097,0.562598,0.186264,0.495186,0.314694,0.076815,0.719549,0.548686,0.204529,0.433335,0.494178,0.264478,0.243059,0.264689,0.263490,0.268942,0.346379,0.372457,0.422964,0.384102,0.490125,0.449754,0.403816,0.0294

# Perform Feature Selection

## 1. Filter Method
<font color=blue>**Mutual Information**</font> between feature **ViolentCrimesPerPop (Y in this case)** and each numeric **X** variables.

**Note:** for any feature features that has **Mutual Information** more than or equal to **0.4**, I will display the text color in <font color=green>**Green**</font> since that mean It has pretty high "amount of information" obtained about **Y** feature through another **X** feature.

In [93]:
for col in df.columns.difference(['ViolentCrimesPerPop']):
    print(f'\033[1m\033[4mCorrelation and Mutual Information between {col} and ViolentCrimesPerPop\033[0m')
    # Calculation Correlation between x and y
    corr = np.corrcoef(df[col], df['ViolentCrimesPerPop'])[0, 1]
    # To calculate the mutual information, we first need to convert the x and y into a 2 dimension histogram
    c_xy = np.histogram2d(df[col], df['ViolentCrimesPerPop'], 20)[0]
    # Use the function called mutual_info_score to calculate the mutual information 
    mi = mutual_info_score(None, None, contingency=c_xy)
    print(("\033[92m" if mi >= 0.4 else "") + "Correlation between X and Y is %.2f"%corr + ("\x1b[0m" if mi >= 0.4 else ""))
    print(("\033[92m" if mi >= 0.4 else "") + "Mutual information=%.2f"%mi + ("\x1b[0m" if mi >= 0.4 else ""))
    print('------------------------------------------------------------------------------')

Correlation and Mutual Information between AsianPerCap and ViolentCrimesPerPop
Correlation between X and Y is -0.16
Mutual information=0.14
------------------------------------------------------------------------------
Correlation and Mutual Information between FemalePctDiv and ViolentCrimesPerPop
Correlation between X and Y is 0.56
Mutual information=0.33
------------------------------------------------------------------------------
Correlation and Mutual Information between HispPerCap and ViolentCrimesPerPop
Correlation between X and Y is -0.24
Mutual information=0.16
------------------------------------------------------------------------------
Correlation and Mutual Information between HousVacant and ViolentCrimesPerPop
Correlation between X and Y is 0.42
Mutual information=0.20
------------------------------------------------------------------------------
Correlation and Mutual Information between LandArea and ViolentCrimesPerPop
Correlation between X and Y is 0.20
Mutual informat

Correlation between X and Y is -0.47
Mutual information=0.26
------------------------------------------------------------------------------
Correlation and Mutual Information between PctIlleg and ViolentCrimesPerPop
Correlation between X and Y is 0.74
Mutual information=0.47
------------------------------------------------------------------------------
Correlation and Mutual Information between PctImmigRec10 and ViolentCrimesPerPop
Correlation between X and Y is 0.29
Mutual information=0.16
------------------------------------------------------------------------------
Correlation and Mutual Information between PctImmigRec5 and ViolentCrimesPerPop
Correlation between X and Y is 0.22
Mutual information=0.14
------------------------------------------------------------------------------
Correlation and Mutual Information between PctImmigRec8 and ViolentCrimesPerPop
Correlation between X and Y is 0.25
Mutual information=0.16
------------------------------------------------------------------

Correlation between X and Y is 0.15
Mutual information=0.14
------------------------------------------------------------------------------
Correlation and Mutual Information between agePct16t24 and ViolentCrimesPerPop
Correlation between X and Y is 0.10
Mutual information=0.16
------------------------------------------------------------------------------
Correlation and Mutual Information between agePct65up and ViolentCrimesPerPop
Correlation between X and Y is 0.07
Mutual information=0.10
------------------------------------------------------------------------------
Correlation and Mutual Information between blackPerCap and ViolentCrimesPerPop
Correlation between X and Y is -0.28
Mutual information=0.19
------------------------------------------------------------------------------
Correlation and Mutual Information between householdsize and ViolentCrimesPerPop
Correlation between X and Y is -0.03
Mutual information=0.11
-----------------------------------------------------------------

## 2. Wrapper Method
I decide to use <font color=blue>**Backward Stepwise Model Selection**</font> which mean we start the model with all features included and we removes one feature on each iteration. The variables that can minimally increase the **Residual Sum of Squares (RSS)** on the data is chosen as the feature to be removed from the model.

In [99]:
# Create linear regression model as the estimator
estimator = LinearRegression()
# From the Filter method, we select 5 features (color in green)
selector = RFE(estimator, 5, step=1)
# Learn from this dataset
targetOutcome = pd.DataFrame(df,columns=['ViolentCrimesPerPop'])
allInputs = pd.DataFrame(df,columns=df.columns.difference(['ViolentCrimesPerPop']))
selector = selector.fit(allInputs, targetOutcome)
# The mask of selected features (which variables are selected)
print(selector.support_)
# Selected features are ranked 1. The variable with the highest rank is the one that is removed first
print(selector.ranking_)
# Print the name of selected features
f = selector.get_support(True) # the most important features
print('\033[1m\033[4mSeledted features from Backward Stepwise Model Selection are: \033[0m')
for f_index in f:
    print(df.columns[f_index])

[False False False False False False False False False False  True False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False  True False False False False False
 False  True False False False False False False False False False  True
 False False False False False False False False False False False False
 False False]
[102   2  88   9  94  89 114  46  15  45   1  14 112  25   1  43  97  77
  44  18  78 108  49  24  34 100  22  52  47  72 117   8   7  95 113  74
 106  38  99  29  62 109  76  12  27  85 101  86  92   4  50  81  64  68
  30  73  54  16  11  56  31  32  33 

## 3. Embedded Method
I will use **LASSO** to select features on this data set.

In [120]:
# penalty of the norm-1 localization
alpha = 0.007

# Learn from this dataset
targetOutcome = pd.DataFrame(df,columns=['ViolentCrimesPerPop'])
allInputs = pd.DataFrame(df,columns=df.columns.difference(['ViolentCrimesPerPop']))

clf = linear_model.Lasso(alpha=alpha)
clf.fit(allInputs, targetOutcome)

print(clf.coef_)
print(clf.intercept_)
print("Sum of square of coefficients = %.2f"%np.sum(clf.coef_**2))

[-0.          0.         -0.          0.          0.          0.
  0.00247801 -0.         -0.          0.          0.          0.
  0.          0.          0.          0.         -0.          0.
 -0.         -0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.          0.          0.          0.
 -0.         -0.          0.20680625  0.          0.          0.
  0.         -0.31050398  0.          0.          0.          0.
  0.          0.         -0.          0.         -0.          0.
  0.          0.          0.         -0.          0.          0.
  0.          0.          0.          0.         -0.         -0.
 -0.         -0.          0.          0.         -0.          0.
  0.         -0.         -0.         -0.          0.         -0.
 -0.          0.          0.          0.          0.          0.
  0.          0.         

[(-0.0, 'AsianPerCap'),
 (0.0, 'FemalePctDiv'),
 (-0.0, 'HispPerCap'),
 (0.0, 'HousVacant'),
 (0.0, 'LandArea'),
 (0.0, 'LemasGangUnitDeploy'),
 (0.002478005250279332, 'LemasPctOfficDrugUn'),
 (-0.0, 'LemasPctPolicOnPatr'),
 (-0.0, 'LemasSwFTFieldOps'),
 (0.0, 'LemasSwFTFieldPerPop'),
 (0.0, 'LemasSwFTPerPop'),
 (0.0, 'LemasSwornFT'),
 (0.0, 'LemasTotReqPerPop'),
 (0.0, 'LemasTotalReq'),
 (0.0, 'MalePctDivorce'),
 (0.0, 'MalePctNevMarr'),
 (-0.0, 'MedNumBR'),
 (0.0, 'MedOwnCostPctInc'),
 (-0.0, 'MedOwnCostPctIncNoMtg'),
 (-0.0, 'MedRent'),
 (0.0, 'MedRentPctHousInc'),
 (0.0, 'MedYrHousBuilt'),
 (0.0, 'NumIlleg'),
 (0.0, 'NumImmig'),
 (0.0, 'NumInShelters'),
 (0.0, 'NumKindsDrugsSeiz'),
 (0.0, 'NumStreet'),
 (0.0, 'NumUnderPov'),
 (0.0, 'OfficAssgnDrugUnits'),
 (0.0, 'OtherPerCap'),
 (-0.0, 'OwnOccHiQuart'),
 (-0.0, 'OwnOccLowQuart'),
 (-0.0, 'OwnOccMedVal'),
 (-0.0, 'PctBSorMore'),
 (-0.0, 'PctBornSameState'),
 (-0.0, 'PctEmplManu'),
 (-0.0, 'PctEmplProfServ'),
 (-0.0, 'PctEmploy'),
 (

In [121]:
print('\033[1m\033[4mSeledted features using LASSO (non zero value of the coefficient) are: \033[0m')
list(zip(clf.coef_, allInputs))

Seledted features using LASSO (non zero value of the coefficient) are: 


[(-0.0, 'AsianPerCap'),
 (0.0, 'FemalePctDiv'),
 (-0.0, 'HispPerCap'),
 (0.0, 'HousVacant'),
 (0.0, 'LandArea'),
 (0.0, 'LemasGangUnitDeploy'),
 (0.002478005250279332, 'LemasPctOfficDrugUn'),
 (-0.0, 'LemasPctPolicOnPatr'),
 (-0.0, 'LemasSwFTFieldOps'),
 (0.0, 'LemasSwFTFieldPerPop'),
 (0.0, 'LemasSwFTPerPop'),
 (0.0, 'LemasSwornFT'),
 (0.0, 'LemasTotReqPerPop'),
 (0.0, 'LemasTotalReq'),
 (0.0, 'MalePctDivorce'),
 (0.0, 'MalePctNevMarr'),
 (-0.0, 'MedNumBR'),
 (0.0, 'MedOwnCostPctInc'),
 (-0.0, 'MedOwnCostPctIncNoMtg'),
 (-0.0, 'MedRent'),
 (0.0, 'MedRentPctHousInc'),
 (0.0, 'MedYrHousBuilt'),
 (0.0, 'NumIlleg'),
 (0.0, 'NumImmig'),
 (0.0, 'NumInShelters'),
 (0.0, 'NumKindsDrugsSeiz'),
 (0.0, 'NumStreet'),
 (0.0, 'NumUnderPov'),
 (0.0, 'OfficAssgnDrugUnits'),
 (0.0, 'OtherPerCap'),
 (-0.0, 'OwnOccHiQuart'),
 (-0.0, 'OwnOccLowQuart'),
 (-0.0, 'OwnOccMedVal'),
 (-0.0, 'PctBSorMore'),
 (-0.0, 'PctBornSameState'),
 (-0.0, 'PctEmplManu'),
 (-0.0, 'PctEmplProfServ'),
 (-0.0, 'PctEmploy'),
 (

***
**Summary:**
1. There are a lot of missing for this data set. Therefore, I need to **replace missing values** with its **Median**.
2. Feature Selection:
    1. **Filter Method** with **Mutual Information** greater or equal to 0.4 as a criterial.
        - Result: Select 5 features from 122 features:
            - PctYoungKids2Par
            - racePctWhite
            - PctFam2Par
            - PctIlleg
            - PctKids2Par
    2. **Wrapper Method** with **Backward Stepwise Model Selection**.
        - Result: Select 5 features from 122 features:
            - numbUrban
            - pctWFarmSelf
            - NumStreet
            - LemasSwFTPerPop
            - PctPolicHisp
    3. **Embedded Method** with **LASSO** and alpha = 0.007.
        - Result: Select 5 features from 122 features (where coefficients not equal to zero):
            - LemasPctOfficDrugUn
            - PctIlleg
            - PctKids2Par
            - pctUrban
            - racePctWhite
3. We can see that each methods return different set of **selected features**. Therefore, we can't rely on only single method or model but we have to try a vareity of ways to find the best approach.
***